# Testing BIBIMBpy

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import agama

agama.setUnits(mass = 1, length = 1, velocity = 1)

In [3]:
import bibimbpy

## create initial conditions

In [4]:
r_ic = 8
phi_ic = 0
z_ic = 0
vz_ic = 0

vr_ic = np.linspace(-100,100,100)
vphi_ic = np.linspace(-400,-100,100)

In [7]:
ic,var1,var2 = bibimbpy.initialize.set_initial_conditions(r_ic,phi_ic,z_ic,vr_ic,vphi_ic,vz_ic)

In [8]:
ic.shape

(10000, 6)

## Generate axisymmetric potential